In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
import fss_rpt

In [2]:
# 환경설정
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.0f}'.format
if not any([s == 'result' for s in os.listdir('.')]): os.mkdir('result')

In [3]:
# 데이터 불러오기
# IFRSDB

# 기타
위험계수 = pd.read_excel('data/위험계수_201912.xlsx')
RBC분류 = pd.read_excel('data/RBC분류.xlsx', sheet_name='자동차')['KICS분류'].to_numpy()
자동차_직전1년마감실적 = fss_rpt.load_rpt('data/업무보고서_202012.xlsx', 'AI060')
자동차_1년전1년마감실적 = fss_rpt.load_rpt('data/업무보고서_201912.xlsx', 'AI060')
자동차_2년전1년마감실적 = fss_rpt.load_rpt('data/업무보고서_201812.xlsx', 'AI060')
자동차_직전1년마감실적.insert(0, 'KICS분류', RBC분류)
자동차_1년전1년마감실적.insert(0, 'KICS분류', RBC분류)
자동차_2년전1년마감실적.insert(0, 'KICS분류', RBC분류)

In [4]:
def 합산비율_계산(df):
    # 'E': '경과보험료', 'IA': '발생손해액', 'QA': '순사업비'

    return df \
        .query('~KICS분류.isna()', engine='python') \
        .groupby('KICS분류')[['E', 'IA', 'QA']].sum() \
        .eval('합산비율 = (IA+QA)/E')[['합산비율']] \
        .reset_index() \
        .rename(columns = {'KICS분류': '보종'})

In [5]:
# 데이터 가공하기
# 위험계수
자동차_위험계수_가공 = 위험계수.query('보종.str.contains("자동차")', engine='python') \
    .assign(보종 = lambda x: x.보종.str.replace(r'\d+(.)', '', regex=True))

# 회사합산비율
자동차_직전1년마감실적_가공 = 합산비율_계산(자동차_직전1년마감실적).rename(columns={'합산비율': '합산비율_직전1년'})
자동차_1년전1년마감실적_가공 = 합산비율_계산(자동차_1년전1년마감실적).rename(columns={'합산비율': '합산비율_1년전1년'})
자동차_2년전1년마감실적_가공 = 합산비율_계산(자동차_2년전1년마감실적).rename(columns={'합산비율': '합산비율_2년전1년'})

자동차_조정위험계수 = 자동차_2년전1년마감실적_가공 \
    .merge(자동차_1년전1년마감실적_가공, on='보종', how='outer') \
    .merge(자동차_직전1년마감실적_가공, on='보종', how='outer') \
    .merge(자동차_위험계수_가공, on='보종', how='outer') \
    .eval('회사합산비율 = (합산비율_직전1년 + 합산비율_1년전1년 + 합산비율_2년전1년)/3') \
    .assign(보험가격조정위험계수 = lambda x: np.fmax(x.보험가격기본위험계수*0.7, x.보험가격기본위험계수+(x.회사합산비율-x.기준합산비율)*0.5))

# 정렬
자동차_순서 = {'개인용자동차(인담보)': 0, '개인용자동차(물담보)': 1, '업무용자동차(인담보)': 2, '업무용자동차(물담보)': 3, '영업용자동차(인담보)': 4, '영업용자동차(물담보)': 5, '기타(자동차)': 6}
자동차_조정위험계수.index = 자동차_조정위험계수['보종'].apply(lambda x: 자동차_순서.get(x)).to_numpy()
자동차_조정위험계수 = 자동차_조정위험계수.sort_index()

In [6]:
# 데이터 출력하기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/자동차_조정위험계수_{now}.xlsx', 'xlsxwriter') as writer:
    자동차_조정위험계수.to_excel(writer, '자동차_조정위험계수', index=False)